# Analise e Previsão de Acidentes de Transito Fatais em Rodovias Federais Brasileiras

Rayron Ferreira

05/03/2025

### Motivação

- impacto economico
- impacto social
- quanto custa por ano os acidentes ?
- qual o impacto com acidentes fatais ?
- reduzir o numero de acidentes de transito e esses impactos
- mitigar/evitar condicoes nas estradas que causam acidentes fatais
- previsao de acidente e gravidade do acidente
- podemos identificar os padroes de como os acidentes fatais ocorrem e os principais fatores?
- alocar melhor os recursos financeiros e humanos nas rodovias federais buscando evitar acidentes fatais

### Objetivos


### Analise

### Palavras-chave

### Sobre o Dataset

### Agradecimentos e Colaborações  

### Referencias

## Visualização e pre-processamento dos dados

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
# from pymongo import MongoClient
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

### Classificação das Variáveis

**Atributos de Identificação**
* **id**: Identificador do acidente.
* **pesid**: Identificador da pessoa envolvida.
* **id_veiculo**: Identificador do veículo envolvido.

**Atributos de Período do Dia**
* **data_inversa**: Data da ocorrência.
* **dia_semana**: Dia da semana da ocorrência.
* **horario**: Horário da ocorrência.
* **fase_dia**: Fase do dia no momento do acidente.

**Atributos de Endereço**
* **uf**: Unidade da Federação.
* **br**: Identificador da BR.
* **km**: Quilômetro do acidente.
* **municipio**: Nome do município.
* **latitude**: Latitude do local do acidente.
* **longitude**: Longitude do local do acidente.
* **regional**: Superintendência regional da PRF.
* **delegacia**: Delegacia da PRF.
* **uop**: Unidade operacional da PRF.

**Atributos de Tráfego**
* **sentido_via**: Sentido da via.
* **tipo_pista**: Tipo de pista.
* **tracado_via**: Descrição do traçado da via.
* **uso_solo**: Características do local do acidente (urbano/rural).

**Atributos Climáticos**
* **condição_meteorologica**: Condição meteorológica no momento do acidente.

**Atributos de Causa e Tipo de Acidente**
* **causa_principal**: Causa principal do acidente.
* **causa_acidente**: Causa presumível do acidente.
* **ordem_tipo_acidente**: Sequência de eventos no acidente.
* **tipo_acidente**: Tipo de acidente.
* **classificação_acidente**: Gravidade do acidente.

**Atributos de Veículo**
* **tipo_veiculo**: Tipo de veículo.
* **marca**: Marca do veículo.
* **ano_fabricacao_veiculo**: Ano de fabricação do veículo.

**Atributos de Pessoas Envolvidas**
* **tipo_envolvido**: Tipo de envolvido no acidente.
* **estado_fisico**: Condição física do envolvido.
* **idade**: Idade do envolvido.
* **sexo**: Sexo do envolvido.

**Atributos de Vítimas**
* **ilesos**: Identifica se o envolvido foi ileso.
* **feridos_leves**: Identifica se o envolvido foi ferido leve.
* **feridos_graves**: Identifica se o envolvido foi ferido grave.
* **mortos**: Identifica se o envolvido foi morto.


### Leitura dos Dados

In [2]:
df_prf_2017_2023 = pd.read_excel('data/acidentes2013-2023.xlsx')

df_prf_2017_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977884 entries, 0 to 977883
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   data_inversa            977884 non-null  object
 1   dia_semana              977884 non-null  object
 2   horario                 977884 non-null  object
 3   uf                      977884 non-null  object
 4   br                      977884 non-null  int64 
 5   km                      977884 non-null  object
 6   municipio               977884 non-null  object
 7   causa_acidente          977884 non-null  object
 8   tipo_acidente           977884 non-null  object
 9   classificacao_acidente  977884 non-null  object
 10  fase_dia                977884 non-null  object
 11  sentido_via             977884 non-null  object
 12  condicao_metereologica  977884 non-null  object
 13  tipo_pista              977884 non-null  object
 14  tracado_via             977884 non-n

In [20]:
df_prf_2024 = pd.read_excel('data/acidentes2024.xlsx')

df_prf_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152203 entries, 0 to 152202
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   data_inversa            152203 non-null  object 
 1   dia_semana              152203 non-null  object 
 2   horario                 152203 non-null  object 
 3   uf                      152203 non-null  object 
 4   br                      152203 non-null  int64  
 5   km                      152203 non-null  object 
 6   municipio               152203 non-null  object 
 7   causa_acidente          152203 non-null  object 
 8   tipo_acidente           152203 non-null  object 
 9   classificacao_acidente  152203 non-null  object 
 10  fase_dia                152203 non-null  object 
 11  sentido_via             152203 non-null  object 
 12  condicao_metereologica  152203 non-null  object 
 13  tipo_pista              152203 non-null  object 
 14  tracado_via         

In [21]:
df_prf_2024.to_json("data/acidentes2024.json", orient="records", indent=4, force_ascii=False)


### Manipulando Texto e Atributos Categóricos

#### Factorize

In [3]:
df_teste = df_prf_2017_2023.copy()

In [4]:
# transforma em numérico, porem os algorítimos de ML, nao podem medir a distancia entre uma categoria e outra
fase_dia_cat = df_teste['fase_dia']
fase_dia_cat_encoded, fase_dia_categories = fase_dia_cat.factorize()

In [10]:
fase_dia_categories

Index(['plena_noite', 'amanhecer', 'pleno_dia', 'anoitecer'], dtype='object')

In [11]:
fase_dia_cat_encoded

array([0, 0, 1, ..., 3, 3, 0], dtype=int64)

#### OneHot Encoder

In [12]:
encoder = OneHotEncoder()
fase_dia_cat_1hot = encoder.fit_transform(fase_dia_cat_encoded.reshape(-1,1))
fase_dia_cat_1hot

<977884x4 sparse matrix of type '<class 'numpy.float64'>'
	with 977884 stored elements in Compressed Sparse Row format>

In [13]:
fase_dia_cat_1hot.toarray()

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

In [16]:
from sklearn.preprocessing import OrdinalEncoder 
ordinal_encoder = OrdinalEncoder()
fase_dia_cat_reshaped = fase_dia_cat.values.reshape(-1, 1)
fase_dia_cat_1hot = ordinal_encoder.fit_transform(fase_dia_cat_reshaped)
fase_dia_cat_1hot

array([[2.],
       [2.],
       [0.],
       ...,
       [1.],
       [1.],
       [2.]])

In [18]:
ordinal_encoder.categories_

[array(['amanhecer', 'anoitecer', 'plena_noite', 'pleno_dia'], dtype=object)]

### Clusterização

### Pre Processamento dos Dados

In [ ]:
# checando variáveis categóricas
variaveis_categoricas = ['municipio', 'uf','causa_acidente','tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'uso_solo', 'estado_fisico', 'tipo_envolvido', 'marca', 'tipo_pista' ]

for variavel in variaveis_categoricas:
  print(variavel, df_prf_2017_2023[variavel].unique())

In [5]:
df_prf_2017_2023 = df_prf_2017_2023.dropna(subset=['ilesos', "feridos_leves", "feridos_graves", "mortos"])

In [6]:
def is_within_brazil(lat, lon):
    return -33.7422 <= lat <= 5.2718 and -73.989 <= lon <= -34.793

def encoding(df):
    label = LabelEncoder()
    for c in df.select_dtypes("object"):
        df[c] = df[c].astype(str)
        df[c]=label.fit_transform(df[c])
    return df

def imputation(df):
    df = df.fillna(df.median())
    df = df.dropna()
    return df

def preprocessing(df):
    df = encoding(df)
    df = imputation(df) 

    return df


def calcular_gravidade(row):
    if row['ilesos'] == 1:
        return 1
    elif row['feridos_leves'] == 1:
        return 2
    elif row['feridos_graves'] == 1:
        return 3
    elif row['mortos'] == 1:
        return 4
    else:
        return 0

def calcular_fase_dia(row):
    if row['fase_dia'] == 'plena_noite':
        return 1
    elif row['fase_dia'] == 'amanhecer':
        return 2
    elif row['fase_dia'] == 'pleno_dia':
        return 3
    elif row['fase_dia'] == 'anoitecer':
        return 4
    
def calcular_condicao_metereologica(row):
    if row['condicao_metereologica'] == 'ceu_claro':
        return 1
    elif row['condicao_metereologica'] == 'vento':
        return 2
    elif row['condicao_metereologica'] == 'sol':
        return 3
    elif row['condicao_metereologica'] == 'nublado':
        return 4
    if row['condicao_metereologica'] == 'garoa_chuvisco':
        return 5
    elif row['condicao_metereologica'] == 'chuva':
        return 6
    elif row['condicao_metereologica'] == 'nevoeiro_neblina':
        return 7
    elif row['condicao_metereologica'] == 'granizo':
        return 8
    elif row['condicao_metereologica'] == 'neve':
        return 9

def calcular_tipo_pista(row):
    if row['tipo_pista'] == 'simples':
        return 1
    elif row['tipo_pista'] == 'dupla':
        return 2
    elif row['tipo_pista'] == 'multipla':
        return 3

    
def calcular_tipo_acidente(row):
    if row['tipo_acidente'] == 'eventos_atipicos':
        return 1
    elif row['tipo_acidente'] == 'danos_eventuais':
        return 2
    elif row['tipo_acidente'] == 'derramamento_de_carga':
        return 3
    elif row['tipo_acidente'] == 'tombamento':
        return 4
    if row['tipo_acidente'] == 'atropelamento_de_pedestre':
        return 5
    elif row['tipo_acidente'] == 'queda_de_ocupante_de_veiculo':
        return 6
    elif row['tipo_acidente'] == 'engavetamento':
        return 7
    elif row['tipo_acidente'] == 'saida_de_leito_carrocavel':
        return 8
    elif row['tipo_acidente'] == 'atropelamento_de_animal':
        return 9
    elif row['tipo_acidente'] == 'colisao_lateral_mesmo_sentido':
        return 10
    if row['tipo_acidente'] == 'colisao_lateral_sentido_oposto':
        return 11
    elif row['tipo_acidente'] == 'colisao_com_objeto':
        return 12
    elif row['tipo_acidente'] == 'colisao_com_objeto_estatico':
        return 13
    elif row['tipo_acidente'] == 'colisao_com_objeto_em_movimento':
        return 14
    if row['tipo_acidente'] == 'colisao_transversal':
        return 15
    elif row['tipo_acidente'] == 'colisao_lateral':
        return 16
    elif row['tipo_acidente'] == 'colisao_traseira':
        return 17
    elif row['tipo_acidente'] == 'colisao_frontal':
        return 18
    elif row['tipo_acidente'] == 'capotamento':
        return 19
    elif row['tipo_acidente'] == 'incendio':
        return 20

In [ ]:
print('Valores únicos ilesos')
for ileso in df_prf_2017_2023["ilesos"].unique():
    print(ileso)

print('Valores únicos feridos_leves')
for ferido_leve in df_prf_2017_2023["feridos_leves"].unique():
    print(ferido_leve)

print('Valores únicos feridos_graves')
for ferido_grave in df_prf_2017_2023["feridos_graves"].unique():
    print(ferido_grave)

print('Valores únicos mortos')
for morto in df_prf_2017_2023["mortos"].unique():
    print(morto)

In [8]:
df_prf_2017_2023['longitude'] = df_prf_2017_2023['longitude'].str.replace(",", ".").astype(float).round(6)
df_prf_2017_2023['latitude'] = df_prf_2017_2023['latitude'].str.replace(",", ".").astype(float).round(6)

df_prf_2017_2023 = df_prf_2017_2023[df_prf_2017_2023.apply(lambda row: is_within_brazil(row['latitude'], row['longitude']), axis=1)]

In [9]:
df_prf_2017_2023['gravidade'] = df_prf_2017_2023.apply(calcular_gravidade, axis=1)
df_prf_2017_2023 = df_prf_2017_2023[df_prf_2017_2023['gravidade'] != 0]

In [10]:
df_prf_2017_2023['tipo_acidente'] = df_prf_2017_2023.apply(calcular_tipo_acidente, axis=1)
df_prf_2017_2023['tipo_pista'] = df_prf_2017_2023.apply(calcular_tipo_pista, axis=1)
df_prf_2017_2023['condicao_metereologica'] = df_prf_2017_2023.apply(calcular_condicao_metereologica, axis=1)
df_prf_2017_2023['fase_dia'] = df_prf_2017_2023.apply(calcular_fase_dia, axis=1)

In [ ]:
df_prf_2017_2023["tipo_acidente"].unique()

In [ ]:
df_prf_2017_2023["tipo_pista"].unique()

In [ ]:
df_prf_2017_2023["condicao_metereologica"].unique()

In [ ]:
df_prf_2017_2023["fase_dia"].unique()

In [11]:
df_prf_2017_2023 = preprocessing(df_prf_2017_2023)

In [ ]:
df_prf_2017_2023.head()

In [13]:
count_class_4, count_class_1, count_class_3, count_class_2 = df_prf_2017_2023['gravidade'].value_counts()

df_class_1 = df_prf_2017_2023[df_prf_2017_2023['gravidade'] == 1]
df_class_2 = df_prf_2017_2023[df_prf_2017_2023['gravidade'] == 2]
df_class_3 = df_prf_2017_2023[df_prf_2017_2023['gravidade'] == 3]
df_class_4 = df_prf_2017_2023[df_prf_2017_2023['gravidade'] == 4]

df_class_1_under = df_class_1.sample(count_class_2,random_state=42)
df_class_4_under = df_class_4.sample(count_class_2,random_state=42)
df_class_3_under = df_class_3.sample(count_class_2,random_state=42)
df_under = pd.concat([df_class_1_under, df_class_2, df_class_3_under, df_class_4_under], axis=0)

df_class_2_over = df_class_2.sample(count_class_1, replace=True, random_state=42)
df_class_3_over = df_class_3.sample(count_class_1, replace=True, random_state=42)
df_class_4_over = df_class_4.sample(count_class_1, replace=True, random_state=42)
df_over = pd.concat([df_class_1, df_class_2_over, df_class_3_over, df_class_4_over], axis=0)

In [14]:
df_to_set = df_over.select_dtypes(include=['float64', 'int64', 'int32'])
df_to_set = df_to_set.drop(["marca", "uso_solo", "modelo_veiculo", "gravidade4", "estado_fisico", "mortos", "feridos_graves", "feridos_leves", "ilesos", "dia_semana", "data_inversa", "horario", "municipio", "classificacao_acidente", "fase_dia", "id_veiculo", "tipo_envolvido"], axis=1)

In [16]:
df_teste = df_prf_2017_2023.copy()
df_teste = df_teste.drop(['classificacao_acidente','ilesos','feridos_leves','feridos_graves', 'mortos', 'estado_fisico', 'marca'], axis=1)

In [20]:
df_teste = df_teste.select_dtypes(include=['float64', 'int64', 'int32'])

## Regressao Logistica

In [56]:
df_logistic_regression = df_teste.copy()

# split X, y
X_lr = df_logistic_regression.drop('gravidade', axis=1)
y_lr = df_logistic_regression['gravidade']

scaler_lr = StandardScaler()
X_scaled_lr = scaler_lr.fit_transform(X_lr)

# split train, test
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_scaled_lr, y_lr, test_size=0.3, random_state=42)

In [ ]:
model_lr = LogisticRegression(max_iter=4000)  # Ajuste 'max_iter' para garantir a convergência
model_lr.fit(X_train_lr, y_train_lr)

In [58]:
y_pred_lr = model_lr.predict(X_test_lr)

In [ ]:
print(accuracy_score(y_test_lr, y_pred_lr))

In [ ]:
confusion_matrix_lr = confusion_matrix(y_test_lr, y_pred_lr)

disp_lr = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_lr, display_labels=model_lr.classes_)
disp_lr.plot(cmap='Greens')
plt.title("Matriz de Confusão")
plt.show()

#Classe Negativa (Verdadeiro Negativo - VN)	VN	FP (Falso Positivo)
#Classe Positiva (Falso Negativo - FN)	FN	VP (Verdadeiro Positivo)

# Verdadeiros Positivos (VP): Casos corretamente classificados como positivos.
# Verdadeiros Negativos (VN): Casos corretamente classificados como negativos.
# Falsos Positivos (FP): Casos negativos classificados erroneamente como positivos (tipo I).
# Falsos Negativos (FN): Casos positivos classificados erroneamente como negativos (tipo II).


In [ ]:
print(classification_report(y_test_lr, y_pred_lr))

In [ ]:
# Interpretação do Gráfico
# Curva mais próxima do canto superior esquerdo:

# Indica um modelo com melhor capacidade de separação entre as classes.
# AUC próximo de 1:

# Significa que o modelo tem excelente desempenho.
# AUC próximo de 0.5:

# Indica que o modelo não tem poder discriminativo (equivalente ao chute aleatório).
# Avaliação prática:

# Se o AUC é alto, mas o modelo ainda comete muitos erros, verifique os dados (desbalanceamento, qualidade) ou ajuste o limiar de decisão.

## floresta


In [21]:
df_rf = df_teste.copy()

# split X, y
X_rf = df_rf.drop('gravidade', axis=1)
y_rf = df_rf['gravidade']

# scaler_rf = StandardScaler()
# X_scaled_rf = scaler_rf.fit_transform(X_rf)

# split train, test
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_rf, y_rf, test_size=0.3, random_state=42)

In [22]:
rf = RandomForestClassifier(
    n_estimators=100,  # Número de árvores na floresta
    max_depth=None,    # Profundidade máxima das árvores
    random_state=42,   # Para resultados reprodutíveis
    class_weight='balanced'  # Ajusta peso das classes automaticamente
)

In [ ]:
rf.fit(X_train_rf, y_train_rf)

In [24]:
y_pred_rf = rf.predict(X_test_rf)

In [ ]:
print("Matriz de Confusão:")

confusion_matrix_rf = confusion_matrix(y_test_rf, y_pred_rf)

disp_rf = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix_rf, display_labels=rf.classes_)
disp_rf.plot(cmap='Greens')
plt.title("Matriz de Confusão")
plt.show()

In [ ]:
print('Accuracy = ', accuracy_score(y_test_rf, y_pred_rf))

In [ ]:
print("\nRelatório de Classificação:")
print(classification_report(y_test_rf, y_pred_rf))

In [ ]:
importances = rf.feature_importances_
for feature, importance in zip(X_rf.columns, importances):
    print(f"{feature}: {importance:.4f}")

In [ ]:
# Converter o array bidimensional para unidimensional
importances = importances.ravel()

# Criar um DataFrame com as features e suas importâncias
importances_df = pd.DataFrame({
    'Feature': X_rf.columns,
    'Importance': importances
})

# Filtrar apenas as features com importância maior que zero
importances_df = importances_df[importances_df['Importance'] > 0]

# Ordenar as features pela importância (do maior para o menor)
importances_df = importances_df.sort_values(by='Importance', ascending=False)

# Selecionar as 10 maiores importâncias
top_10_importances = importances_df.head(30)

# Plotar o gráfico de barras
plt.figure(figsize=(10, 6))
plt.barh(top_10_importances['Feature'], top_10_importances['Importance'], color='green')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Top 30 Feature Importances')
plt.gca().invert_yaxis()  # Inverter o eixo Y para ter as maiores importâncias no topo
plt.show()

### Rede Neural

In [15]:
# Importar bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

In [16]:
features = ['idade', 'horario', 'latitude', 'longitude', 'modelo_veiculo', 'km', 'id_veiculo', 'data_inversa', 'municipio', 'ano_fabricacao_veiculo', 'causa_acidente', 'tipo_veiculo', 'marca_veiculo', 'br', 'dia_semana', 'tracado_via', 'uf', 'uso_solo', 'tipo_envolvido', 'sentido_via', 'sexo']

In [17]:
# 1. Carregar os dados
# Substitua 'seu_dataset.csv' pelo caminho para o seu arquivo
df_rna = df_prf.copy()
df_rna = df_rna.select_dtypes(include=['float64', 'int64', 'int32'])

#df_rna = df_rna[colunas_importanes]

In [18]:
df_rna = df_rna.dropna()

In [ ]:
df_rna

In [20]:
# Suponha que 'target' é a coluna de rótulos e o restante são as features
X_rna = df_rna[features]                  # Features
y_rna = df_rna['gravidade']              # Classe

In [21]:
# 2. Pré-processamento dos dados
# Dividir o conjunto em treino e teste
X_train_rna, X_test_rna, y_train_rna, y_test_rna = train_test_split(X_rna, y_rna, test_size=0.3, random_state=42)

In [91]:
# Normalizar as features (padrão necessário para redes neurais)
# scaler_rna = StandardScaler()
# X_train_rna = scaler_rna.fit_transform(X_train_rna)
# X_test_rna = scaler_rna.transform(X_test_rna)

In [22]:
# Se o problema for de classificação com múltiplas classes, fazer one-hot encoding
num_classes_rna = len(np.unique(y_rna))  # Número de classes

# Ajustar os rótulos para começar de 0 (se necessário)
y_train_rna = y_train_rna - y_train_rna.min()
y_test_rna = y_test_rna - y_test_rna.min()

y_train_rna = to_categorical(y_train_rna, num_classes_rna)
y_test_rna = to_categorical(y_test_rna, num_classes_rna)

In [23]:
# 3. Construir a rede neural
model_rna = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_rna.shape[1],)),  # Camada oculta com 64 neurônios
    Dense(32, activation='relu'),                                  # Camada oculta com 32 neurônios
    Dense(num_classes_rna, activation='softmax' if num_classes_rna > 2 else 'sigmoid')  # Camada de saída
])

In [24]:
# 4. Compilar o modelo
model_rna.compile(
    optimizer='adam',                      # Otimizador eficiente
    loss='categorical_crossentropy' if num_classes_rna > 2 else 'binary_crossentropy',  # Função de perda
    metrics=['accuracy']                   # Métrica de avaliação
)

In [ ]:
# 5. Treinar o modelo
history = model_rna.fit(
    X_train_rna, y_train_rna,
    epochs=50,                # Número de épocas
    batch_size=42,            # Tamanho do lote
    validation_split=0.3,     # Usar 20% do treino para validação
    verbose=1                 # Mostrar progresso
)

In [ ]:
# 6. Avaliar o modelo
loss_rna, accuracy_rna = model_rna.evaluate(X_test_rna, y_test_rna)
print(f"\nLoss: {loss_rna:.4f}, Accuracy: {accuracy_rna:.4f}")

In [ ]:
# Fazer previsões
y_pred_rna = model_rna.predict(X_test_rna)
if num_classes_rna > 2:
    y_pred_classes_rna = np.argmax(y_pred_rna, axis=1)  # Converter para rótulos de classe
    y_true_classes_rna = np.argmax(y_test_rna, axis=1)
else:
    y_pred_classes_rna = (y_pred_rna > 0.5).astype(int).flatten()
    y_true_classes_rna = y_test_rna

In [ ]:
# 7. Métricas de desempenho
print("\nMatriz de Confusão:")
print(confusion_matrix(y_true_classes_rna, y_pred_classes_rna))

In [ ]:
print("\nRelatório de Classificação:")
print(classification_report(y_true_classes_rna, y_pred_classes_rna))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

# Carregar os dados
df = df_rna.copy()  # Substitua pelo seu dataset

# Pré-processamento
# Selecione features relevantes e a variável alvo
features = ['idade', 'horario', 'latitude', 'longitude', 'modelo_veiculo', 'km', 'id_veiculo', 'data_inversa', 'municipio', 'ano_fabricacao_veiculo', 'causa_acidente', 'tipo_veiculo', 'marca_veiculo', 'br', 'dia_semana', 'tracado_via', 'uf', 'uso_solo', 'sentido_via', 'sexo']
target = 'gravidade'

# One-hot encoding para variáveis categóricas
# df = pd.get_dummies(df, columns=features, drop_first=True)

# Separar X (entrada) e y (saída)
X = df.drop(columns=[target])
y = df[target]

# Divisão treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalização dos dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Criar e treinar a rede neural
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

# Previsões
y_pred = mlp.predict(X_test)

# Avaliação
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
df[features].info()

normalizacao nos dados de treino

cross-validation